**Zepeng Xiao, zx1004** <br>

# **Using Monte Carlo** <br>

(a) solution: <br>

In the risk neutral measure, the price of the call should be the current value of the expectation of call's payoff.

Let $p, v$ be the call's payoff and price, respectively: $\displaystyle v = e^{-r(T - t)}\mathbb{E}[p \ \vert \ \mathcal{F}_t]$ 

- When $\displaystyle S_u \leq B$ for some $\displaystyle u \in (t, T) \Rightarrow \mathbb{1}_{S_u > B, \ \forall \, u \in (t, T)} = 0$, <br>
hence the payoff of this call is $\displaystyle p = 0 = (S_T - K)_{+}\mathbb{1}_{S_u > B, \ \forall \, u \in (t, T)}$ <br>

- When $\displaystyle S_u > B, \ \forall \, u \in (t, T) \Rightarrow \mathbb{1}_{S_u > B, \ \forall \, u \in (t, T)} = 1$, <br>
hence the payoff of this call is $\displaystyle p = (S_T - K)_{+} = (S_T - K)_{+}\mathbb{1}_{S_u > B, \ \forall \, u \in (t, T)}$ <br>

Hence $\displaystyle (S_T - K)_{+}\mathbb{1}_{S_u > B, \ \forall \, u \in (t, T)}$ is exactly the payoff of this call.

Therefore, the price of this option is: $$\displaystyle v = e^{-r(T - t)}\mathbb{E}[(S_T - K)_{+}\mathbb{1}_{S_u > B, \ \forall \, u \in (t, T)} \ \vert \ \mathcal{F}_t]$$

In [1]:
# Question (b)

import numpy as np
import pandas as pd

np.random.seed(11078211)

# the function that generate a geometric Brownian Motion path
# @input:
#     T: time to maturity
#     N: the number of time step in this path
#     r: drift rate
#     sigma: volatility
#     S0: initial stock price
# @return: the geometric Brownian Motion path
def MCpath(T, N, r, sigma, S0):
    dt = T * 1. / N # compute time interval
    path = np.zeros(N + 1) # initialize an array to store the path
    path[0] = S0 # set initial price
    for i in range(N):
        # apply Euler method to generate price for each time step
        dW = np.random.normal(0, np.sqrt(dt))
        dS = r * path[i] * dt + sigma * path[i] * dW
        path[i + 1] = path[i] + dS
    return path

path = MCpath(T= 1, N = 252, r = .02, sigma = .20, S0 = 100)
print(pd.DataFrame(path.reshape(1, 253)))

     0          1          2          3          4          5         6    \
0  100.0  98.356201  96.455484  96.517981  96.862312  98.575936  97.52034   

         7          8           9    ...         243         244         245  \
0  97.641496  99.580532  101.301502  ...  113.186652  113.490267  116.104558   

        246         247         248         249         250         251  \
0  117.0164  114.945605  118.642378  119.729023  121.205131  120.894002   

          252  
0  121.764142  

[1 rows x 253 columns]


(b): <br>

The program that computes one trajectory of $S_t$ for $t < T$ is showed above.

Also, one sample trajectory computed by this program is showed above. <br>



In [2]:
# Question (c)

# the function that compute the payoff of the barrier call option
# given a stock price path follows geometric Brownian Motion
# @input:
#     path: the stock price path follows geometric Brownian Motion
#     B: the barrier
#     K: the strike price
# @return: the payoff of barrier call option
def MCpathValue(path, B, K):
    # if barrier is reached, the payoff will be zero 
    if (np.all(path > B)):
        return max(path[-1] - K,0)
    # if not, compute payoff as the vanilla European call
    else:
        return 0

# price the path that generated in question (b)
print("The payoff of the call computed by the path generated in (b) is:", 
      MCpathValue(path = path, B = 80, K = 110))

The payoff of the call computed by the path generated in (b) is: 11.764141755851014


(c): <br>

The program that takes the list of values computed in (b) and returns the payoff of the option is showed above.

Also, the payoff of the sample trajectory generated in (b) is showed above. <br>



In [3]:
# Question (d)

# the function that price the barrier call option by Monte Carlo
# @input:
#     T: time to maturity
#     N: the number of time step in this path
#     r: drift rate
#     sigma: volatility
#     S0: initial stock price
#     B: the barrier
#     K: the strike price
#     Np: number of path that will be gererated
# @return: the payoff of barrier call option
def MCcallValue(T, N, r, sigma, S0, B, K, Np):
    sum = 0
    # generate paths, and compute the payoff of each path
    for j in range(Np):
        samplePath = MCpath(T, N, r, sigma, S0)
        sum += MCpathValue(samplePath, B, K)
    # the discount value of the average of the payoff will be the price
    return sum * np.exp(-r * T) / Np

print("The price of call when applying Monte Carlo method is: ", 
      MCcallValue(1, 252, .02, .20, 100, 80, 110, 100000))

The price of call when applying Monte Carlo method is:  4.883948533685356


(d): <br>

The program that applying Monte Carlo to price the barrier call option is mentioned above.

When using $10^5$ trajectories, we know that the price of call is 4.883948533685356.